In [2]:
!pip install selenium

# 필요한 패키지 불러오기

In [3]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import pickle

# 호텔별 URL을 가져오기

In [4]:
# url 함수
def make_url(Url):
    import time
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome()
    driver.get(Url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    click1 = driver.find_element(By.XPATH,'//*[@id="__next"]/div/header/section/div/div[1]/section[2]/div/div/div/div/div[2]/div/div[1]/span[1]/span')
    click1.click()
    time.sleep(6)
    click2 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[1]')
    click2.click()
    click3 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[2]')
    click3.click()
    time.sleep(2)
    click4 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/footer/div/button[2]')
    click4.click()
    time.sleep(2)
    #무한 스크롤
    
    time.sleep(1)
    
    SCROLL_PAUSE_TIME = 1.5
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height  

    soup = BeautifulSoup(html, 'html.parser')
    data = soup.select("section > div > div > a")
        
    for d in data:
        hotelurl.append(d.attrs['href'])
    # driver.close()
  

In [6]:
hotelurl = []
make_url("https://www.yanolja.com/hotel/r-900582?advert=AREA&topAdvertiseMore=1")

# 리뷰 수집

In [14]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome()

hotel = []
star = []
reviews = []
rev_sum_pos = []
rev_sum_neg = []
columns = ['hotel','star' ,'reviews', 'Review Summary Pos', 'Review Summary Neg','url' ]   


for url in hotelurl:
    cnt = 0
    SCROLL_PAUSE_TIME = 0.5
    time.sleep(1)
    Url = "https://www.yanolja.com"+url
    driver.get(Url)
    WebDriverWait(driver, 10)
    # try:
    # # 요소 찾기 시도
    try : 
        rev_sum_pos.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text)
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except NoSuchElementException:
        rev_sum_pos.append("")
    try:   
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except TimeoutException:
        pass
    try :    
        rev_sum_neg.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text) 
    except NoSuchElementException:
        rev_sum_neg.append("") 
    # except NoSuchElementException:
    # # 요소가 없을 경우 공백 문자열 설정
    #     rev_sum_pos = ''
    #     rev_sum_neg = ''
    
    hotel.append(driver.find_element(By.CSS_SELECTOR,'#__next > div > div > main > article > div.css-1cc3d9 > div > div.css-1e56347 > div.css-11vo59c > h1').text)
    
    # 리뷰 페이지 클릭 (Click review page)
    try:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-xa55ms'))
        )
    except NoSuchElementException :
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    except TimeoutException:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    element2.click()
    # 무한 스크롤
    import time
    time.sleep(1)
    
    # SCROLL_PAUSE_TIME = 1.5
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "css-stl2cn")))
    element.click()
    
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[text()='평점 낮은순']")))
    element.click()
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    max = int(driver.find_element(By.CLASS_NAME,"css-12rhouj").text.replace("(","").replace(")",""))
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
    
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if max > 1000:
            if(cnt>20):
                break
            cnt +=1
        if new_height == last_height:
            break
    
        last_height = new_height
    
    # 스크롤 끝나고 페이지 elements 가져오기
    star.append(float(driver.find_element(By.CLASS_NAME, 'css-u41ekg').text))
    soup = bs(driver.page_source, "lxml")
    
    # 특정 클래스 이름을 가진 모든 요소 찾기
    elements = soup.find_all('p')
    temp = ""
    # 데이터 추출 
    review = driver.find_elements(By.CLASS_NAME,"css-c92dc4")
    for i in review:
        temp += i.text
    reviews.append(temp)     
    
df = pd.DataFrame({"Hotel" : hotel, "Star" : star ,"Reviews" : reviews, "Review Pos" : rev_sum_pos,"Review Neg" : rev_sum_neg,"Url": hotelurl})   
df["Url"] ="https://www.yanolja.com"+df["Url"]

In [16]:
with open('서울.pickle', 'wb') as f:
    pickle.dump(df, f)

In [18]:
df

,Hotel,Star,Reviews,Review Pos,Review Neg,Url
0,역삼 아르누보씨티 호텔 앤 레지던스,4.5,진짜 최악이에요.. 청소 하나도 안되어있고 먼지의 소굴임\n그리고 관리도 제대로 안...,"넓고 깨끗한 객실을 제공하며, 가성비가 뛰어난 것으로 평가됩니다. 침대와 침구는 편...","저렴한 가격으로 강남 지역에서 머무를 수 있는 장점이 있지만, 전반적으로 청결 상태...",https://www.yanolja.com/hotel/10042496
1,호텔 소울하다,4.2,싱크대에 뭉친 머리카락도 있고 바닥에도 머리카락도 많았네요 전체적으로 청소도 안된 ...,"강남 접근성이 우수하며, 청결한 객실과 친절한 서비스가 특징입니다. 대부분의 객실에...","강남역 인근에 위치해 있으며, 조용한 환경을 제공합니다. 객실은 대체로 깨끗하지만 ...",https://www.yanolja.com/hotel/3001688
2,글래드 마포,4.8,"장난하나요? 주차비 1,2만원은 서울권에서 비지니스 호텔에서는 내니까 이해합니다\n...","공덕역 인근에 위치해 있어 접근성이 뛰어나며, 주변에 다양한 식당과 편의시설이 있어...",위치가 공덕역 바로 앞에 있어 접근성이 좋으며 주변에 맛집도 다양하게 분포되어 있습...,https://www.yanolja.com/hotel/3012800
3,노보텔 앰배서더 서울 용산,4.7,전혀 추천하지 않음진짜 ^^1bar 개더럽다 그냥 내가 사진 동영상 까려다 참는다\...,"용산역과 아이파크몰과 가까워 접근성이 우수하며, 내부는 깔끔하고 청결한 상태를 유지...",용산역과 가까운 위치로 대중교통 접근성이 좋습니다. 전체적으로 시설은 깔끔하지만 객...,https://www.yanolja.com/hotel/3015391
4,몬드리안 서울 이태원(Mondrian Seoul Itaewon),4.6,샤워실에서는 물도 안내려가고.. 바퀴벌레도 발견했어요.. 최악이에요..5성급 호텔인...,"이숙소는 가성비가 뛰어난 5성급 호텔로, 실내 및 야외 수영장, 헬스장, 다양한 편...","IA주 이태원에 위치하고 있으며, 직원들의 친절한 서비스와 다양한 편의 시설이 인상...",https://www.yanolja.com/hotel/1000109193
...,...,...,...,...,...,...
66,라마다 서울 신도림,4.8,호텔에 먼지가 많고 시트엔 구멍나고 전체적으로 호텔 자체가 노후화됐고 관리도 제대로...,"넓고 깨끗한 객실과 편안한 침구류로 호평받고 있습니다. 주차 편의성과 청결도, 친절...","청결도는 비교적 좋은 편이나, 노후화된 시설과 높은 가격으로 불만이 제기되고 있습니...",https://www.yanolja.com/hotel/3012441
67,르컬렉티브 강남 예술의전당 로이움,4.8,방음이 안됩니다.. 근처 볼일있어서 숙박했는데 새볔에 옆방소리때문에 깸.. 밑에 편...,깨끗하고 편안한 환경을 제공합니다. 주변에 편의점과 교통편이 좋아 접근성이 뛰어난 ...,"체크인 절차가 간편하고, 주변에 편의점이 있어 접근성이 좋습니다. 객실은 전반적으로...",https://www.yanolja.com/hotel/10054310
68,더 리센츠 동대문 호텔,4.2,일부러 트윈으로 신청했는데 더블 침대 들어갈 자리에 싱글 침대 두개 붙여서 넣어놓고...,가격 대비 뛰어난 가성비와 청결함이 특징입니다. 푹신한 침구로 편안한 숙면이 가능하...,"깔끔한 내부와 좋은 접근성을 제공하지만, 다수의 이용객이 방음 문제와 냄새에 대한 ...",https://www.yanolja.com/hotel/3009564
69,글래드 강남 코엑스센터,4.6,공사소음때문에 전혀 쉴수가없어서 몇시간 뒤에 바로 체크아웃했습니다별 한개도 과분한 ...,"전반적으로 이 숙소는 청결하고 편리한 위치를 갖춘 곳으로, 삼성역에서 가까워 접근성...","삼성역 바로 앞에 위치하여 접근성이 뛰어나며, 주변에 편의시설이 잘 갖춰져 있습니다...",https://www.yanolja.com/hotel/3012015
